# Day 12: Garden Groups

## Import libraries

In [1]:
import copy

## Import data

In [ ]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - Grid map represents garden plots.
# - 1 garden plot: Represented by 1 distinct letter in the grid = 1 type of plant.
# - 1 region: Represented by SAME garden plots touching horizontally/vertically (E.g. 'AAAA').
# - Area: Represented by the number of garden plots in a region (E.g. Area of region 'AAAA' = 4).
# - Perimeter: Represented by the *number of sides* of SAME garden plots in a region that do NOT touch another garden plot in the SAME region.
# - NOTE: EACH garden plot = a square with 4 sides = 1 grid block.
# - NOTE: 1 LONE garden plot = 1 region with area = 1.
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-12_input-test.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

file_data = file_data.split(" ")

# print(file_data)
# ====================================================================================================================

## Helper functions

In [6]:
"""
NOTE to future self (and other devs): Yes, I know you are expecting some 'def ...' python helper function here, but this was actually the ULTIMATE helper! :o So, long story short, I didn't have hours of time to sit and spend hours writing the code to solve this quite fun problem (I actually enjoy these maze problems, they just sometimes take time to convert the solutions to code, well for me at least). Therefore, I sat and understood the challenge and then wrote a well-constructed prompt about the challenge to input into AI and requested it to generate a python function solution to solve the problem for me and it worked! (I literally did not change any logic from the function that it generated for me - I neatened the code up and changed a few variables and return values to my liking and style but I left the main logic untouched :D). Find the entire example prompt below: :p

You are an experience python developer with over 20 years of experience in solving coding challenges. I would like you to please understand the following challenge and then generate a python function in order to solve it:

I would like you to generate a python function that accepts a string grid populated with different uppercase letters. Each distinct upper case letter in the grid is known as a garden plot = a single grid block letter with 4 sides. Now, the entire grid consists of different regions of garden plots. 1 region is represented by the same garden plots (letters) touching each other each horiziontally and/or vertically. If however, a garden plot is found on its own in the grid (E.g. No other same garden plot letters are touching it horizontally or vertically), then this lone garden plot creates a region of its own. An area of a region = the total number of garden plots (letters) in the region. A perimeter of a region = the number of sides of same garden plots in a region that do NOT touch each other (E.g. a region 'AAAA' is a region of 'A' garden plots that exists in a straight line from left to right with area = 4 because there are 4 'A's and perimeter = 10 because the number of outter garden plot sides that do not touch each other within the region = 10). A price of a region is calculated as: region area * region perimeter. Let's see an example grid below:

AAAA
BBCD
BBCC
EEEC

In the example above, The area of a region is simply the number of garden plots the region contains. The above map's type A, B, and C plants are each in a region of area 4. The type E plants are in a region of area 3; the type D plants are in a region of area 1. Each garden plot is a square and so has four sides. The perimeter of a region is the number of sides of garden plots in the region that do not touch another garden plot in the same region. The type A and C plants are each in a region with perimeter 10. The type B and E plants are each in a region with perimeter 8. The lone D plot forms its own region with perimeter 4. Therefore, region A has a price 4 * 10 = 40, region B has price 4 * 8 = 32, region C has price 4 * 10 = 40, region D has price 1 * 4 = 4, and region E has price 3 * 8 = 24. So, the total price for this first example is 140. 

However, do keep in mind that garden plots of the same letter can also appear in multiple separate regions and regions can appear within other regions. Second example below:

OOOOO
OXOXO
OOOOO
OXOXO
OOOOO

The example grid above contains 5 regions (1 containing all of the 'O' garden plots and the other 4 each containing a single 'X' garden plot). The four X regions each have area = 1 and perimeter = 4. The region containing 21 'O' garden plots is more complicated - in addition to its outer edge contributing a perimeter of 20, its boundary with each 'X' region contributes an additional 4 to its perimeter, for a total perimeter of 36. In this second example above, the region with all of the 'O' garden plots has price 21 * 36 = 756, and each of the four smaller X regions has price 1 * 4 = 4, for a total price of 772 (756 + 4 + 4 + 4 + 4).

The generated python function should please print out a message for each region found in the grid in the following format: "Region x with price: region_x_area * region_x_perimeter", where 'x' is the current region being traversed through.

NOTE: Although this took me about 10-20min to firsly understand the problem and then write the prompt, I am glad I did it because this helped me understand how to better structure and write prompts to AI (Prompt Eng.). Also, 30min of writing a prompt vs. 2/3 hours of coding? >>>>


"""

'\nYou are an experience python developer with over 20 years of experience in solving coding challenges. I would like you to please understand the following challenge and then generate a python function in order to solve it:\n\nI would like you to generate a python function that accepts a string grid populated with different uppercase letters. Each distinct upper case letter in the grid is known as a garden plot = a single grid block letter with 4 sides. Now, the entire grid consists of different regions of garden plots. 1 region is represented by the same garden plots (letters) touching each other each horiziontally and/or vertically. If however, a garden plot is found on its own in the grid (E.g. No other same garden plot letters are touching it horizontally or vertically), then this lone garden plot creates a region of its own. An area of a region = the total number of garden plots (letters) in the region. A perimeter of a region = the number of sides of same garden plots in a regio

In [7]:
def calculate_region_price(grid):
    if not grid or not grid[0]:
        return 0

    rows, cols = len(grid), len(grid[0])
    visited = set()
    total_price = 0

    def dfs(r, c, letter):
        stack = [(r, c)]
        area = 0
        perimeter = 0
        
        while stack:
            x, y = stack.pop()
            if (x, y) in visited:
                continue
            
            visited.add((x, y))
            area += 1
            
            # Check all four directions
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                
                if 0 <= nx < rows and 0 <= ny < cols:
                    if grid[nx][ny] == letter:
                        stack.append((nx, ny))
                    else:
                        perimeter += 1  # Different letter or empty space contributes to perimeter
                else:
                    perimeter += 1  # Out of bounds contributes to perimeter
        
        return area, perimeter

    for r in range(rows):
        for c in range(cols):
            if (r, c) not in visited:
                letter = grid[r][c]
                area, perimeter = dfs(r, c, letter)
                price = area * perimeter
                total_price += price
                print(f"Region {letter} with price: {area} * {perimeter} = {price}")

    print(f"Total price: {total_price}")
    return total_price

# Example usage
grid1 = [
    "AAAA",
    "BBCD",
    "BBCC",
    "EEEC"
]

grid2 = [
    "OOOOO",
    "OXOXO",
    "OOOOO",
    "OXOXO",
    "OOOOO"
]

grid3 = [
    "RRRRIICCFF",
    "RRRRIICCCF",
    "VVRRRCCFFF",
    "VVRCCCJFFF",
    "VVVVCJJCFE",
    "VVIVCCJJEE",
    "VVIIICJJEE",
    "MIIIIIJJEE",
    "MIIISIJEEE",
    "MMMISSJEEE"
]

# print("Example 1:")
# calculate_region_price(grid1)

# print("\nExample 2:")
# calculate_region_price(grid2)

print("\nExample 3:")
calculate_region_price(grid3)


Example 3:
Region R with price: 12 * 18 = 216
Region I with price: 4 * 8 = 32
Region C with price: 14 * 28 = 392
Region F with price: 10 * 18 = 180
Region V with price: 13 * 20 = 260
Region J with price: 11 * 20 = 220
Region C with price: 1 * 4 = 4
Region E with price: 13 * 18 = 234
Region I with price: 14 * 22 = 308
Region M with price: 5 * 12 = 60
Region S with price: 3 * 8 = 24
Total price: 1930


1930

In [37]:

# ====================================================================================================================

## Part 1

In [ ]:
# *** [PART 1] ***
# ! PROBLEM: You're about to settle near a complex arrangement of garden plots when some Elves ask if you can lend a hand. They'd like to set up fences around EACH region of garden plots, but they can't figure out how much fence they need to order or how much it will cost. They hand you a map (your puzzle input) of the garden plots.
# - NOTE: The *price* of a fence required for a region = (region area * region perimeter).
# ! TODO: Calculate the total price of fencing ALL regions on your map.
# ====================================================================================================================
# ! Create a deep (independent) copy of the data, such that changes made to the copy do not affect the original data to still test/re-run Part 1/2 with the correct INITIAL (and not modified) data
# - NOTE: Not using a deep copy will modify the original data after running Part 1/2, therefore no correct output will be calculated anymore.
var = copy.deepcopy(file_data)




# ====================================================================================================================

## Part 2

In [ ]:
# *** [PART 2] ***
# ! PROBLEM: xxx
# ! TODO: xxx
#====================================================================================================================
# ! Create a deep (independent) copy of the data, such that changes made to the copy do not affect the original data to still test/re-run Part 1/2 with the correct INITIAL (and not modified) data
# - NOTE: Not using a deep copy will modify the original data after running Part 1/2, therefore no correct output will be calculated anymore.



